# Preprocessing Washington Allocation data for WaDEQA upload.
- Date Updated: 04/01/2020
- Purpose:  To preprocess the Washington data into one master file for simple DataFrame creation and extraction

Useful Links to Data:
- The Data - Geographic Water Information System (GWIS)Data from the WA stat: https://fortress.wa.gov/ecy/gispublic/DataDownload/wr/GWIS_Data/
- Data dictionary - https://fortress.wa.gov/ecy/gispublic/DataDownload/wr/GWIS_Data/GWIS_Data_Dictionary/
- Public website   - https://ecology.wa.gov/Water-Shorelines/Water-supply/Water-rights

In [8]:
# Needed Libararies
import os
import numpy as np
import pandas as pd
import geopandas as gpd # the library that lets us read in shapefiles
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

from pyproj import Transformer, transform
transformer = Transformer.from_proj(2927, 4326)  # A trick to drastically optimize the Transformer of pyproj.
# Washignton projection = EPSG:2927. WGS84 projection used by WaDE 2.0 = epsg:4326.

In [9]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Washington/WaterAllocation/RawInputData"
os.chdir(workingDir)

## POD Data

In [10]:
# # Dataframe creation
# df_1 = pd.read_csv(d_pointFile, encoding = "ISO-8859-1") #Input
# df_2 = pd.read_csv(D_Point_WR_DocFile, encoding = "ISO-8859-1") #Input
# df_3 = pd.read_csv(Person_Plus_EXTRACT_FromWRTSnotGWISFile, encoding = "ISO-8859-1") #Input

In [12]:
# Input File, contains PoD info
d_pointFile = "D_PointTable.zip"
df_1 = pd.read_csv(d_pointFile, encoding = "ISO-8859-1")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in df_1:
    df_1['WaDEUUID'] = "waD" + df_1.index.astype(str)
    df_1.to_csv('D_PointTable.csv', index=False)

print(len(df_1))
df_1.head()

155880


C:\Users\rjame\AppData\Local\Temp\ipykernel_19320\3694054506.py:3: DtypeWarning: Columns (7,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv(d_pointFile, encoding = "ISO-8859-1")


,OBJECTID,D_Point_ID,D_Point_Ty,Location_C,Assoc_FL,Misc_CD,Position_W,Active_DT,Inactive_D,Update_TD,Update_Use,Comment_DS,Created_TD,Created_Us,POINT_X,POINT_Y,WaDEUUID
0,1,200801.0,WL,U,N,,S,NaN,NaN,3/28/2013 9:58:00,"ECY\DKRO461""""",NaN,NaN,NaN,1.665873e+06,454923.236269,waD0
1,2,200889.0,MW,G,Y,,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.816741e+06,456744.055134,waD1
2,3,200890.0,MW,G,Y,,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.813848e+06,456733.006272,waD2
3,4,201092.0,WL,U,Y,,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.571475e+06,457350.130694,waD3
4,5,201191.0,MW,G,Y,,S,NaN,NaN,6/11/2010 11:28:19,"ECY\DKRO461""""",NaN,NaN,NaN,1.715392e+06,402037.784298,waD4


In [14]:
# Input File, Bridge table
D_Point_WR_DocFile = "D_Point_WR_Doc.zip"
df_2 = pd.read_csv(D_Point_WR_DocFile, encoding = "ISO-8859-1")
print(len(df_2))
df_2.head()

175196


C:\Users\rjame\AppData\Local\Temp\ipykernel_19320\3198338825.py:3: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv(D_Point_WR_DocFile, encoding = "ISO-8859-1")


,OBJECTID,D_Point_WR_Doc_ID,D_Point_ID,WR_Doc_NR,WR_Doc_ID,Active_DT,Inactive_DT,Update_TD,Update_User_ID,Created_TD,Created_User_ID
0,1566,NaN,204903.0,G4-26991C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1567,NaN,204904.0,G4-26830C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1568,NaN,204905.0,G4-29018C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1569,NaN,204906.0,G4-29018C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1570,NaN,204907.0,G4-25116C,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Input File, Contains water use and owner info
Person_Plus_EXTRACT_FromWRTSnotGWISFile = "Person_Plus_EXTRACT_FromWRTSnotGWIS.zip"
df_3 = pd.read_csv(Person_Plus_EXTRACT_FromWRTSnotGWISFile, encoding = "ISO-8859-1")
print(len(df_3))
df_3.head()

301470


,ÃÂ¯ÃÂ»ÃÂ¿OID_,OBJECTID_1,WaRecId,WaRecId_1,WR_Doc_ID,WaRecPhaseId,PartyRoleTypeCode,PersonLastOrOrganizationNM,PersonFirstNM,PersonMINM,PersonAddressLine1AD,PersonAddressLine2AD,PersonAddressLine3AD,PersonAddressCityAD,PersonAddressStateCode,PersonAddressZipCodeAD,WaRecRCWClassTypeCode,EcologyRegionCode,WaRecPrimaryNumber,PriorityDate,WaRecProcessStatusTypeCode,WaRecClaimTypeCode,WaRecPhaseTypeCode,WaRecPhaseStageTypeCode,InstantaneousQuantity,AnnualVolumeQuantity,IrrigatedAreaQuantity,InstantaneousUnitCode,PurposeOfUseTypeCodes
0,1,260791,2144735,2144735,2144735,-1,Primary,Bengen Farms Dba Juniper Road Farms,NaN,NaN,591 Bengen Lane,NaN,NaN,Pasco,WA,99301,groundwater,ERO,G3-28565,11/2/1988 0:00,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,267032,2144672,2144672,2144672,-1,Primary,Bengen Farms Dba Juniper Road Farms,NaN,NaN,591 Bengen Lane,NaN,NaN,Pasco,WA,99301,groundwater,ERO,G3-24977,7/24/1976 0:00,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,177316,2142777,2142777,2142777,-1,Primary,Creston Town,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,groundwater,ERO,G3-*00582S,9/1/1942 0:00,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,260780,2145783,2145783,2145783,-1,Primary,Bengen Farms Dba Juniper Road Farms,NaN,NaN,591 Bengen Lane,NaN,NaN,Pasco,WA,99301,groundwater,ERO,G3-25099(A),11/4/1976 0:00,Active,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,275652,6500899,6500899,6500899,-1,Primary,Farmland Reserve Inc,NaN,NaN,6716 W Rio Grande Ave,PO Box 2308,NaN,Pasco,WA,99302-2308,surfaceWater,CRO,CS4-16571(A1)@1,2/17/2015 0:00,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Merging dataframes into one, using left-join.
df = pd.DataFrame()
df = pd.merge(df_1, df_2, left_on='D_Point_ID', right_on='D_Point_ID', how='left') # Joinning PoD data
df = pd.merge(df,   df_3, left_on='WR_Doc_ID', right_on='WR_Doc_ID', how='left') # Joinning PoD data

df = df.drop_duplicates().replace(np.nan, "").reset_index(drop=True)
print(len(df))
df.head(1)

198075


,OBJECTID_x,D_Point_ID,D_Point_Ty,Location_C,Assoc_FL,Misc_CD,Position_W,Active_DT_x,Inactive_D,Update_TD_x,Update_Use,Comment_DS,Created_TD_x,Created_Us,POINT_X,POINT_Y,WaDEUUID,OBJECTID_y,D_Point_WR_Doc_ID,WR_Doc_NR,WR_Doc_ID,Active_DT_y,Inactive_DT,Update_TD_y,Update_User_ID,Created_TD_y,Created_User_ID,Ã¯Â»Â¿OID_,OBJECTID_1,WaRecId,WaRecId_1,WaRecPhaseId,PartyRoleTypeCode,PersonLastOrOrganizationNM,PersonFirstNM,PersonMINM,PersonAddressLine1AD,PersonAddressLine2AD,PersonAddressLine3AD,PersonAddressCityAD,PersonAddressStateCode,PersonAddressZipCodeAD,WaRecRCWClassTypeCode,EcologyRegionCode,WaRecPrimaryNumber,PriorityDate,WaRecProcessStatusTypeCode,WaRecClaimTypeCode,WaRecPhaseTypeCode,WaRecPhaseStageTypeCode,InstantaneousQuantity,AnnualVolumeQuantity,IrrigatedAreaQuantity,InstantaneousUnitCode,PurposeOfUseTypeCodes
0,1,200801.0,WL,U,N,,S,,,3/28/2013 9:58:00,"ECY\DKRO461""""",,,,1665873.058973,454923.236269,waD0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
# For converting projection latitude.
def assignLat(colrowValueLat, colrowValueLong):
    if colrowValueLat == '' or pd.isnull(colrowValueLat):
        lat = ""
    else:
        lat, long = transformer.transform(colrowValueLat, colrowValueLong)
    return lat

# For converting projection longitude.
def assignLong(colrowValueLat, colrowValueLong):
    if colrowValueLong == '' or pd.isnull(colrowValueLong):
        long = ""
    else:
        lat, long = transformer.transform(colrowValueLat, colrowValueLong)
    return long

df['in_Latitude'] = df.apply(lambda row: assignLat(row['POINT_X'], row['POINT_Y']), axis=1)
df['in_Longitude'] = df.apply(lambda row: assignLong(row['POINT_X'], row['POINT_Y']), axis=1)

In [ ]:
#Changing datatype of used date fields. 
df['PriorityDate'] = pd.to_datetime(df['PriorityDate'], errors = 'coerce')
df['PriorityDate'] = pd.to_datetime(df["PriorityDate"].dt.strftime('%m/%d/%Y'))

In [ ]:
import re

def assignOwner(valueFirst, valueMid, valueLast):
    #--- First Name ---
    if valueFirst == "" or pd.isnull(valueFirst):
        FirstName = ""
    else:
        FirstName = str(valueFirst).strip()
        
    #--- Midile Initial ---
    if valueMid == "" or pd.isnull(valueMid):
        MidName = ""
    else:
        MidName = str(valueMid).strip()
    
    #--- Last Name ---
    if valueLast == "" or pd.isnull(valueLast):
        LastName = ""
    else:
        LastName = str(valueLast).strip()

    if LastName == "":
        outlist = LastName + FirstName + MidName
    else:
        outlist = LastName + ", " + FirstName + " "+ MidName
        
    outlist = re.sub("[$@&.;,/\)(-]", "", outlist).strip()
    
    return outlist


df['Owner'] = df.apply(lambda row: assignOwner(row['PersonFirstNM'],
                                               row['PersonMINM'],
                                               row['PersonLastOrOrganizationNM']), axis=1)
df['Owner'].unique()

In [ ]:
#Manually filling in empty class code with ‘Unspecified’ value.
def assignWaRecRCWClassTypeCode(colValue):
    if colValue == "" or pd.isnull(colValue):
        outlist = "Unspecified"
    else:
        outlist = colValue.strip()
    return outlist


df['WaRecRCWClassTypeCode'] = df.apply(lambda row: assignWaRecRCWClassTypeCode(row['WaRecRCWClassTypeCode']), axis=1)
df['WaRecRCWClassTypeCode'].unique()

In [ ]:
# For creating AllocationAmount
def assignAllocationAmount(colrowValueIQ, colrowValueUC):
    if colrowValueIQ == '' or pd.isnull(colrowValueIQ):
        outVal = ""
    elif colrowValueIQ <= 0 or pd.isnull(colrowValueIQ):
        outVal = 0
    else:
        MultiFactor = 1.0
        gpmcfsUnit = colrowValueUC.strip()
        if gpmcfsUnit == 'GPM':
            MultiFactor = 0.00222800926
        elif gpmcfsUnit == 'GPD':
            MultiFactor = 1.0 / 646317.0
        try:
            outVal = MultiFactor * colrowValueIQ
        except:
            outVal = colrowValueIQ
    return outVal

df['in_AllocationFlow_CFS'] =  df.apply(lambda row: assignAllocationAmount(row['InstantaneousQuantity'], row['InstantaneousUnitCode']), axis=1)
df['in_AllocationFlow_CFS'].unique()

In [ ]:
# # Create output dataframe for ground water
# columnslist = [   
#     ### Water Source Info ###
#     "in_WaterSourceTypeCV",
    
#     ### Site Info ###
#     "in_CoordinateAccuracy",
#     "in_Latitude",
#     "in_Longitude",
#     "in_SiteTypeCV",
#     "in_PODorPOUSite",
    
#     ### AllocationAmount_fact Info ###
#     "in_AllocationFlow_CFS",
#     "in_AllocationLegalStatusCV",
#     "in_AllocationNativeID",
#     "in_AllocationOwner",
#     "in_AllocationPriorityDate",
#     "in_AllocationTypeCV",
#     "in_AllocationVolume_AF",
#     "in_BeneficialUseCategory",
#     "in_IrrigatedAcreage"]

# dfPOD = pd.DataFrame(columns=columnslist, index=df.index)

In [ ]:
#############################################################################################
# Data Assessment UUID
dfPOD['WaDEUUID'] = df['WaDEUUID']

#WaterSource
dfPOD['in_WaterSourceTypeCV'] = df['WaRecRCWClassTypeCode']
                                    
#Site
dfPOD['in_CoordinateAccuracy'] = df['Location_C']
dfPOD['in_Latitude'] = df['in_Latitude']
dfPOD['in_Longitude'] = df['in_Longitude']
dfPOD['in_SiteNativeID'] = "POD" + df['D_Point_ID'].astype(str)
dfPOD['in_SiteTypeCV'] = df['D_Point_Ty']
dfPOD['in_PODorPOUSite'] = "POD"

#AllocationAmount_fact
dfPOD['in_AllocationFlow_CFS'] = df['in_AllocationFlow_CFS']
dfPOD['in_AllocationLegalStatusCV'] = df['WaRecProcessStatusTypeCode'].astype(str)
dfPOD['in_AllocationNativeID'] = df['WR_Doc_ID'].astype(str)
dfPOD['in_AllocationOwner'] = df['Owner'].astype(str)
dfPOD['in_AllocationPriorityDate'] = df['PriorityDate']
dfPOD['in_AllocationTypeCV'] = df['WaRecPhaseTypeCode']
dfPOD['in_AllocationVolume_AF'] = df['AnnualVolumeQuantity']
dfPOD['in_BeneficialUseCategory'] = df['PurposeOfUseTypeCodes'].astype(str)
dfPOD['in_IrrigatedAcreage'] = df['IrrigatedAreaQuantity']

dfPOD = dfPOD.drop_duplicates().replace(np.nan, "").reset_index(drop=True)
print(len(dfPOD))
dfPOD

## POU Data

In [ ]:
# Input File, contains POU info
pouInput = "WA_POU_Input.csv" 
df_1u = pd.read_csv(pouInput)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in df_1u:
    df_1u['WaDEUUID'] = "waU" + df_1u.index.astype(str)
    df_1u.to_csv('WA_POU_Input.csv', index=False)

print(len(df_1u))
df_1u.head(1)

In [ ]:
# Merging dataframes into one, using left-join.
# df_u1 and df3
dfu = pd.DataFrame()
dfu = pd.merge(df_1u, df_3, left_on='WR_DOC_ID', right_on='WR_Doc_ID', how='left')

dfu = dfu.drop_duplicates().replace(np.nan, "").reset_index(drop=True)
print(len(dfu))
dfu.head(1)

In [ ]:
#Changing datatype of used date fields. 
dfu['PriorityDate'] = pd.to_datetime(dfu['PriorityDate'], errors = 'coerce')
dfu['PriorityDate'] = pd.to_datetime(dfu["PriorityDate"].dt.strftime('%m/%d/%Y'))

In [ ]:
def assignOwner(valueFirst, valueMid, valueLast):
    #--- First Name ---
    if valueFirst == "" or pd.isnull(valueFirst):
        FirstName = ""
    else:
        FirstName = str(valueFirst).strip()
        
    #--- Midile Initial ---
    if valueMid == "" or pd.isnull(valueMid):
        MidName = ""
    else:
        MidName = str(valueMid).strip()
    
    #--- Last Name ---
    if valueLast == "" or pd.isnull(valueLast):
        LastName = ""
    else:
        LastName = str(valueLast).strip()

    if LastName == "":
        outlist = LastName + FirstName + MidName
    else:
        outlist = LastName + ", " + FirstName + " "+ MidName
        
    outlist = re.sub("[$@&.;,/\)(-]", "", outlist).strip()
    
    return outlist


dfu['Owner'] = dfu.apply(lambda row: assignOwner(row['PersonFirstNM'], row['PersonMINM'], row['PersonLastOrOrganizationNM']), axis=1)
dfu['Owner'].unique()

In [ ]:
#Manually filling in empty class code with ‘unknown’ value.
def assignWaRecRCWClassTypeCode(colValue):
    if colValue == "" or pd.isnull(colValue):
        outlist = "Unspecified"
    else:
        outlist = colValue.strip()
    return outlist


dfu['WaRecRCWClassTypeCode'] = dfu.apply(lambda row: assignWaRecRCWClassTypeCode(row['WaRecRCWClassTypeCode']), axis=1)
dfu['WaRecRCWClassTypeCode'].unique()

In [ ]:
# For creating AllocationAmount
def assignAllocationAmount(colrowValueIQ, colrowValueUC):
    if colrowValueIQ == '' or pd.isnull(colrowValueIQ):
        outVal = ""
    elif colrowValueIQ <= 0 or pd.isnull(colrowValueIQ):
        outVal = 0
    else:
        MultiFactor = 1.0
        gpmcfsUnit = colrowValueUC.strip()
        if gpmcfsUnit == 'GPM':
            MultiFactor = 0.00222800926
        elif gpmcfsUnit == 'GPD':
            MultiFactor = 1.0 / 646317.0
        try:
            outVal = MultiFactor * colrowValueIQ
        except:
            outVal = colrowValueIQ
    return outVal

dfu['in_AllocationFlow_CFS'] =  dfu.apply(lambda row: assignAllocationAmount(row['InstantaneousQuantity'], row['InstantaneousUnitCode']), axis=1)
dfu['in_AllocationFlow_CFS'].unique()

In [ ]:
# Creating WaDE Custom site native ID for easy site identificaiion
# ----------------------------------------------------------------------------------------------------

# Create temp SiteNativeID dataframe of unique site.
def assignSiteUUID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEWA_S" + string1
    return outstring

dfSiteNativeID = pd.DataFrame()
dfSiteNativeID['in_Latitude'] = dfu['Latitude']
dfSiteNativeID['in_Longitude'] = dfu['Longitude']
dfSiteNativeID = dfSiteNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfSiteNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfSiteNativeID['in_SiteNativeID'] = dftemp.apply(lambda row: assignSiteUUID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom site native ID
def retrieveSiteNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfSiteNativeID.loc[(dfSiteNativeID['in_Latitude'] == A) & 
                                (dfSiteNativeID['in_Longitude'] == B), 'in_SiteNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfu['in_SiteNativeID'] = dfu.apply(lambda row: retrieveSiteNativeID(row['Latitude'], row['Longitude']), axis=1)
dfu['in_SiteNativeID'].unique()

In [ ]:
# # Create output dataframe for ground water
# columnslist = [   
#     ### Water Source Info ###
#     "in_WaterSourceName",
#     "in_WaterSourceTypeCV",
    
#     ### Site Info ###
#     "in_CoordinateAccuracy",
#     "in_Latitude",
#     "in_Longitude",
#     "in_SiteTypeCV",
#     "in_PODorPOUSite",
    
#     ### AllocationAmount_fact Info ###
#     "in_AllocationFlow_CFS",
#     "in_AllocationLegalStatusCV",
#     "in_AllocationNativeID",
#     "in_AllocationOwner",
#     "in_AllocationPriorityDate",
#     "in_AllocationTypeCV",
#     "in_AllocationVolume_AF",
#     "in_BeneficialUseCategory",
#     "in_IrrigatedAcreage"]

# dfPOU = pd.DataFrame(columns=columnslist, index=dfu.index)

In [ ]:
#############################################################################################
# Data Assessment UUID
dfPOU['WaDEUUID'] = dfu['WaDEUUID']

#WaterSource
dfPOU['in_WaterSourceTypeCV'] = dfu['WaRecRCWClassTypeCode']
                                    
#Site
dfPOU['in_CoordinateAccuracy'] = ""
dfPOU['in_Latitude'] = dfu['Latitude']
dfPOU['in_Longitude'] = dfu['Longitude']
dfPOU['in_SiteNativeID'] = "POU" + dfu['in_SiteNativeID'].astype(str)
dfPOU['in_SiteTypeCV'] = "Unspecified"
dfPOU['in_PODorPOUSite'] = "POU"

#AllocationAmount_fact
dfPOU['in_AllocationFlow_CFS'] = dfu['in_AllocationFlow_CFS']
dfPOU['in_AllocationLegalStatusCV'] = dfu['WaRecProcessStatusTypeCode'].astype(str)
dfPOU['in_AllocationNativeID'] = dfu['WR_Doc_ID'].astype(str)
dfPOU['in_AllocationOwner'] = dfu['Owner'].astype(str)
dfPOU['in_AllocationPriorityDate'] = df['PriorityDate']
dfPOU['in_AllocationTypeCV'] = dfu['WaRecPhaseTypeCode']
dfPOU['in_AllocationVolume_AF'] = dfu['AnnualVolumeQuantity']
dfPOU['in_BeneficialUseCategory'] = dfu['PurposeOfUseTypeCodes'].astype(str)
dfPOU['in_IrrigatedAcreage'] = dfu['IrrigatedAreaQuantity']

dfPOU = dfPOU.drop_duplicates().replace(np.nan, "").reset_index(drop=True)
print(len(dfPOU))
dfPOU

## Concatenate POD & POU

In [ ]:
# Concatenate
frames = [dfPOD, dfPOU]
dfout = pd.concat(frames)
dfout = dfout.replace(np.nan, "").drop_duplicates()
dfout = dfout.replace("nan", "").drop_duplicates()

print(len(dfout))
dfout

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEWA_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = dfout['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceTypeCV'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_WaterSourceNativeID'] = dfout.apply(lambda row: retrieveWaterSourceNativeID(row['in_WaterSourceTypeCV']), axis=1)
dfout['in_WaterSourceNativeID'].unique()

## Shapefile Data
- For attaching gemetry to csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
dfPoUshapetemp = gpd.read_file('shapefile/WA_PoU2.shp')
dfPoUshapetemp.head(3)

In [ ]:
# Creating WaDE Custom site native ID for easy site identificaiion
# ----------------------------------------------------------------------------------------------------

# Create temp SiteNativeID dataframe of unique site.
def assignSiteUUID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEWA_S" + string1
    return outstring

dfSiteNativeID = pd.DataFrame()
dfSiteNativeID['in_Latitude'] = dfPoUshapetemp['Latitude']
dfSiteNativeID['in_Longitude'] = dfPoUshapetemp['Longitude']
dfSiteNativeID = dfSiteNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfSiteNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfSiteNativeID['in_SiteNativeID'] = dftemp.apply(lambda row: assignSiteUUID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom site native ID
def retrieveSiteNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfSiteNativeID.loc[(dfSiteNativeID['in_Latitude'] == A) & 
                                (dfSiteNativeID['in_Longitude'] == B), 'in_SiteNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfPoUshapetemp['in_SiteNativeID'] = dfPoUshapetemp.apply(lambda row: retrieveSiteNativeID( row['Latitude'], row['Longitude']), axis=1)
dfPoUshapetemp.head(2)

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['in_SiteNativeID'].astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

## Export Outputs

In [ ]:
#Exporting to Finished File
dfout.to_csv('P_WashingtonMaster.csv', index=False)  # The output
dfPoUshape.to_csv('P_WashingtonGeometry.csv', index=False) # The output geometry.

In [ ]:
dfPOD['in_AllocationLegalStatusCV'] = df['WaRecProcessStatusTypeCode'].astype(str)
dfPOU['in_AllocationTypeCV'] = dfu['WaRecPhaseTypeCode']

In [9]:
df['WaRecPhaseTypeCode'].unique()

array(['', 'ChangeROE', 'NewApp', 'ChangeApplication', 'Certificate',
       'Claim', 'SupersedingCertificate', 'TemporaryDonation',
       'ClaimAmendment', 'Permit', 'SupersedingPermit', 'ROE',
       'AdjudicatedCertificate', 'CertificateOfChange',
       'QuincyBasinPermit', 'SupersedingQuincyBasinPermit',
       'TemporaryUse', 'MitigatedPermit', 'ShortTerm'], dtype=object)

In [14]:
df1 = df[['WaRecProcessStatusTypeCode', 'WaRecPhaseTypeCode']].copy()
df1 = df1.drop_duplicates().reset_index(drop=True).sort_values(by=['WaRecProcessStatusTypeCode', 'WaRecPhaseTypeCode'])
df1

,WaRecProcessStatusTypeCode,WaRecPhaseTypeCode
0,,
5,Active,
16,Active,AdjudicatedCertificate
4,Active,Certificate
17,Active,CertificateOfChange
3,Active,ChangeApplication
1,Active,ChangeROE
6,Active,Claim
9,Active,ClaimAmendment
40,Active,MitigatedPermit
